In [1]:
%matplotlib osx
#qt4
# inline
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os

import pandas as pd
def sec_since_basedate(t64, basedate):
    """ given a numpy datetime 64 object, and a datetime basedate, 
        return seconds since basedate"""
    
    t_series = pd.Series(t64) - basedate
    t = np.fromiter((dt.total_seconds() for dt in t_series), dtype='float64')
    return t

from glmtools.io.glm import GLMDataset, fix_event_locations, fix_unsigned

/Users/ebruning/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
filename = '/data/LCFA-production/OR_GLM-L2-LCFA_G16_s20171161230400_e20171161231000_c20171161231027.nc'
glm = GLMDataset(filename)
# print glm.dataset

In [3]:
from lmatools.lasso.energy_stats import TimeSeriesPolygonLassoFilter
from datetime import datetime, timedelta

In [4]:
# epoch = datetime(2000,1,1,12,0,0)
# tmin = epoch + timedelta(seconds=glm.dataset.event_time_offset.data.min())
# tmax = epoch + timedelta(seconds=glm.dataset.event_time_offset.data.max())
# # tmax = tmin + timedelta(seconds=60)

# # lat0, lat1 = 25.0, 32.0
# # lon0, lon1 = -93.0, -88.0

lat0, lat1 = 31.50, 37.1
lon0, lon1 = -104.00, -99.00

good = ((glm.dataset.flash_lat < lat1) & (glm.dataset.flash_lat > lat0) &
 (glm.dataset.flash_lon < lon1) & (glm.dataset.flash_lat > lon0)).data

# polys = (((lon0, lat0), (lon0, lat1), (lon1, lat1),  (lon1, lat0),),
#         )

# lasso = TimeSeriesPolygonLassoFilter(coord_names=('flash_lon', 'flash_lat'),
#                                      time_edges=(tmin, tmax),
#                                      time_key = 'flash_time_offset_of_first_event', #not strictly everything
#                                      basedate = epoch,
#                                      polys=polys
#                                     )

flash_ids_LMA = glm.dataset.flash_id[good].data
print(flash_ids_LMA)

[6361 6522 6567]


In [5]:
fl_LMA = glm.get_flashes(flash_ids_LMA)
print(fl_LMA.flash_id.data)
print(fl_LMA.group_parent_flash_id.data)
print('-----')
print(np.unique(fl_LMA.group_id.data))
print(np.unique(fl_LMA.event_parent_group_id.data))
# t=glm.dataset.flash_energy[lasso.time_key]
# a=glm.dataset.flash_energy

[6361 6522 6567]
[6361 6361 6361 6361 6361 6361 6361 6522 6522 6522 6522 6522 6522 6522 6522
 6522 6522 6522 6522 6522 6522 6522 6522 6522 6522 6522 6522 6522 6522 6522
 6522 6522 6522 6522 6522 6522 6522 6522 6522 6522 6522 6567 6567 6567 6567
 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567
 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567
 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567
 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567
 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567 6567]
-----
[47021190 47021191 47021203 47021214 47021216 47021218 47021220 47023345
 47023352 47023364 47023367 47023368 47023369 47023370 47023371 47023372
 47023373 47023374 47023375 47023376 47023377 47023378 47023380 47023381
 47023382 47023384 47023387 47023388 47023389 47023390 47023391 47023393
 47023400 47023401 47023403 47023404 47023406 47023407 47023410 47023412
 4702341

In [6]:
# # Doesn't work because good[in_time] in library makes a copy instead of a view
# # mask = lasso.filter_mask(glm.dataset.flash_energy)

# good = np.zeros(a.shape, dtype=bool)
# for (t0, t1), poly_target in lasso.poly_lookup.iteritems():
#     # make sure this is only one-side inclusive to eliminate double-counting
#     in_time = (t >= t0) & (t < t1)
#     reduced = a[in_time]
#     if reduced.size > 0:
#         inpoly = poly_target.filter_mask(reduced)
#         # accept anything in this polygon at this time
#         # use the data attribute to get a numpy boolean array instead 
#         # of the xarray wrapper
#         # if you don't, it makes a copy of `good` and doesn't do the 
#         # in-place boolean mask operation
#         good[in_time.data] |= inpoly


In [7]:
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature

latlon_cs = ccrs.Geodetic()

def no_transform(lons, lats):
    return lons, lats

def plot_fge(flash_data, ax_ev, transform=no_transform, mpl_transform=None):
    """ transform is a function accepting two arrays lon, lat and 
        returning coordinates  x, y to pe plotted on ax_ev.
        
        mpl_transform may be used to pass a transform arguemnt to matplotlib
        as is of commonly used with cartopy plots
        """
#     print(flash_data.number_of_events)

    ev_lats = fix_unsigned(flash_data.event_lat)#.data
    ev_lons = fix_unsigned(flash_data.event_lon)#.data
#     ev_lats, ev_lons = fix_event_locations(flash_data.event_lat, flash_data.event_lon, is_xarray=True)
    ev_time = flash_data.event_time_offset.data
    ev_rad = fix_unsigned(flash_data.event_energy)#.data
    gr_lat = flash_data.group_lat.data
    gr_lon = flash_data.group_lon.data
    gr_rad = fix_unsigned(flash_data.group_energy)#.data)
    fl_lat = flash_data.flash_lat.data
    fl_lon = flash_data.flash_lon.data
    fl_rad = fix_unsigned(flash_data.flash_energy)# .data
    fl_area = fix_unsigned(flash_data.flash_area)#.data

    gr_x, gr_y = transform(gr_lon, gr_lat)
    fl_x, fl_y = transform(fl_lon, fl_lat)
    ev_x, ev_y = transform(ev_lons, ev_lats)
    
    
    kwargs = {}
    if mpl_transform is not None:
        kwargs['transform'] = mpl_transform
    
    ax_ev.scatter(gr_x, gr_y, c=gr_rad, marker='o', 
                  s=100/4, edgecolor='black', cmap='gray_r', 
                  vmin=glm.energy_min, vmax=glm.energy_max, 
                  zorder=100, label='groups', **kwargs)
    ax_ev.scatter(ev_x, ev_y, c=ev_rad, marker='s', 
                  s=16/4, edgecolor='black', cmap='gray', 
                  vmin=glm.energy_min, vmax=glm.energy_max, 
                zorder=101, label='events', **kwargs)
    ax_ev.scatter(fl_x, fl_y, c='r', marker='x', 
                  s=100/4, 
                  zorder=102, label='flashes',
                  **kwargs) #, vmin=glm.energy_min, vmax=glm.energy_max)
#     ax_ev.legend()
    
    
def plot_glm(glm, good, ax, add_groups_events=False, max_area=None):
    """ Plot flashes where good True. good is a boolen array with length that equals the number of flashes"""
    flash_data=glm.dataset
    
    flash_area = fix_unsigned(flash_data.flash_area)

    if max_area is not None:
        area_mask = (flash_area< max_area).data
        good = good & area_mask
    
    flash_ids = glm.dataset.flash_id[good]
    
    fl_lat = flash_data.flash_lat[good].data
    fl_lon = flash_data.flash_lon[good].data
    fl_rad = flash_data.flash_energy[good].data
    fl_area = flash_area[good].data


    if add_groups_events==True:
        for fl_id in flash_ids:
            plot_fge(glm.get_flashes([int(fl_id.data)]), ax, mpl_transform=latlon_cs)
        ax.set_title("Flashes (x), Groups (o), Events (.), max area={0}".format(max_area))
    else:
        area_max = fl_area.max()
        area_scale = 0.16 # 2500.0/area_max # about 0.16 for a large population of MCS flashes
        print(area_scale)
        ax.set_title('Flashes, colored by radiance and sized by flash area, max area {0}'.format(max_area))
        ax.scatter(fl_lon, fl_lat, c=fl_rad, s=fl_area*area_scale, edgecolor='black', marker='o', cmap='gray', alpha=0.5, transform=latlon_cs) #, vmin=glm.energy_min, vmax=glm.energy_max)


plt.figure(figsize=(11,8.5))
# ax = plt.axes(projection=ccrs.Geostationary(central_longitude=-95))
ax = plt.axes(projection=ccrs.PlateCarree())
# ax.coastlines()
# ax.gridlines(draw_labels=True)
ax.set_extent((lon0, lon1, lat0, lat1))
# ax.set_global()

# http://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-1-states-provinces/
states = NaturalEarthFeature(category='cultural', scale='50m', facecolor='none',
                             name='admin_1_states_provinces_shp')
# http://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-populated-places/
roads = NaturalEarthFeature(category='cultural', scale='10m', facecolor='none',
                             name='roads')
ax.add_feature(roads, linewidth=.5, color='gray')
ax.add_feature(states, linewidth=1.0)
ax.coastlines('50m', linewidth=0.8)


# two_group_max_area = 200.0 #km^2
# plot_glm(glm, good, ax, max_area=two_group_max_area)
plot_glm(glm, good, ax, add_groups_events=True)#, max_area=two_group_max_area)



plt.show()



In [8]:
flash_data=glm.dataset.flash_time_offset_of_first_event[good]
print(flash_data)

<xarray.DataArray 'flash_time_offset_of_first_event' (number_of_flashes: 3)>
array(['2017-04-26T07:30:40.902000000-0500',
       '2017-04-26T07:30:52.740000000-0500',
       '2017-04-26T07:30:56.234000000-0500'], dtype='datetime64[ns]')
Coordinates:
    flash_id                          (number_of_flashes) int16 6361 6522 6567
    flash_time_offset_of_first_event  (number_of_flashes) datetime64[ns] 2017-04-26T12:30:40.902000 ...
    flash_time_offset_of_last_event   (number_of_flashes) datetime64[ns] 2017-04-26T12:30:41.210000 ...
    flash_lat                         (number_of_flashes) float32 33.991 ...
    flash_lon                         (number_of_flashes) float32 -101.691 ...
    product_time                      datetime64[ns] 2017-04-26T12:30:40
    lightning_wavelength              float32 777.4
    group_time_threshold              float32 0.0
    flash_time_threshold              float32 3.33
    lat_field_of_view                 float32 0.0
    lon_field_of_view          

/Users/ebruning/anaconda/lib/python2.7/site-packages/matplotlib/artist.py:221: MatplotlibDeprecationWarning: This has been deprecated in mpl 1.5, please use the
axes property.  A removal date has not been set.
  warnings.warn(_get_axes_msg, mplDeprecation, stacklevel=1)


In [9]:
# 20 168 209
# Flash ID 6567 has lots of groups … 78!

print(glm.dataset.flash_id[209])

print((glm.dataset.group_parent_flash_id == 6567).sum())

<xarray.DataArray 'flash_id' ()>
array(6567, dtype=int16)
Coordinates:
    flash_id                          int16 6567
    flash_time_offset_of_first_event  datetime64[ns] 2017-04-26T12:30:56.234000
    flash_time_offset_of_last_event   datetime64[ns] 2017-04-26T12:30:57.256000
    flash_lat                         float32 34.1859
    flash_lon                         float32 -101.607
    product_time                      datetime64[ns] 2017-04-26T12:30:40
    lightning_wavelength              float32 777.4
    group_time_threshold              float32 0.0
    flash_time_threshold              float32 3.33
    lat_field_of_view                 float32 0.0
    lon_field_of_view                 float32 -89.5
Attributes:
    long_name:  product-unique lightning flash identifier
    _Unsigned:  true
    units:      1
<xarray.DataArray 'group_parent_flash_id' ()>
array(78)
Coordinates:
    product_time          datetime64[ns] 2017-04-26T12:30:40
    lightning_wavelength  float32 777.4
    

## LMA

In [10]:
# %matplotlib qt4
from brawl4d.brawl4d import B4D_startup, redraw
import os
import numpy as np

In [11]:
data_path = '/data/LCFA-production/LMArealtime/'
lma_file = os.path.join(data_path, 'LYLOUT_170426_120000_3600.dat.gz') # WTLMA only


mcom_lat, mcom_lon =  33.581908, -101.880804
from datetime import datetime
panels = B4D_startup(basedate=datetime(2017,4,26), ctr_lat=mcom_lat, ctr_lon=mcom_lon)



In [12]:
geosys = panels.cs.geoProj
mapproj = panels.cs.mapProj

# ka1_radial = mapproj.fromECEF(*ka1radarsys.toECEF( np.asarray((0.0, radar_dist)), 
#                                                   np.asarray((ka1_az, ka1_az)), 
#                                                   np.asarray((0.0, 0.0))
#                                                ))
# ka2_radial = mapproj.fromECEF(*ka2radarsys.toECEF( np.asarray((0.0, radar_dist)), 
#                                                   np.asarray((ka2_az, ka2_az)), 
#                                                   np.asarray((0.0, 0.0))
#                                                ))
def plot_ka(ax):
    ax.plot(ka1_radial[0]*1.0e-3, ka1_radial[1]*1.0e-3, 'k')
    ax.annotate('Ka1', xy=(ka1_radial[0][0]*1.0e-3, ka1_radial[1][0]*1.0e-3))
    ax.plot(ka2_radial[0]*1.0e-3, ka2_radial[1]*1.0e-3, 'k')
    ax.annotate('Ka2', xy=(ka2_radial[0][0]*1.0e-3, ka2_radial[1][0]*1.0e-3))
        
    ax.axis((-5, 75, -40, 40))

# plot_ka(panels.panels['xy'])


In [13]:
import matplotlib.pyplot as plt; plt.show()

In [14]:
from brawl4d.LMA.controller import LMAController
lma_ctrl = LMAController()

from matplotlib.cm import get_cmap
# cmap = get_cmap('cubehelix_r')
# scatter_kwargs = {'cmap':cmap}
d, post_filter_brancher, scatter_ctrl, charge_lasso = lma_ctrl.load_dat_to_panels(panels, lma_file, mask_length=6)#,  
                                                                                   #scatter_kwargs=scatter_kwargs)

In [15]:
panels.panels['tz'].axis((12*3600 + 30*60 + 40, 12*3600 + 31*60, 1, 10))
panels.panels['xy'].axis((-60, 60, 0, 120))



(-60, 60, 0, 120)

In [16]:
from brawl4d.LMA.widgets import LMAwidgetController
from IPython.display import display
from brawl4d.LMA.controller import LMAController

lma_tools = LMAwidgetController(panels, lma_ctrl, scatter_ctrl, charge_lasso, d)
display(lma_tools.tools_popup)

In [17]:
flash_ids = (6361, 6522, 6567)

In [18]:


def gen_flashes(glm, flash_ids):
    """ Generate flashes one at a time. Inefficient and not recommended. Use get_flashes on the whole list."""
    for flash_id in flash_ids:
        yield glm.get_flashes([flash_id])

def transform_ll_to_map(lons, lats):
    alts = np.zeros_like(lons)
    geosys = panels.cs.geoProj
    mapproj = panels.cs.mapProj
    x,y,z = mapproj.fromECEF(*geosys.toECEF( lons,lats,alts))    
    return x*1.0e-3, y*1.0e-3



def plot_fge_tz(flash_data, ax, basedate, z=0.0):
    """ Plot GLM data in on a time-height axis, assuming z in km.
        Time is plotted as seconds relative to basedate.
    """
    
    ev_z = z
    gr_z = z
    fl_z = z
    
    ev_rad = fix_unsigned(flash_data.event_energy)#.data
    gr_rad = fix_unsigned(flash_data.group_energy)#.data
    fl_rad = fix_unsigned(flash_data.flash_energy)#.data
    fl_area = fix_unsigned(flash_data.flash_area)#.data
    
    # flashes
    t = sec_since_basedate(flash_data.flash_time_offset_of_first_event.data, panels.basedate)
    z = np.zeros_like(t) + fl_z
    ax.scatter(t, z, marker='x', 
                  s=100/4, 
                  zorder=102, label='flashes',)
        
    # groups
    t = sec_since_basedate(flash_data.group_time_offset.data, panels.basedate)
    z = np.zeros_like(t) + gr_z
    print(t)
    print(z)
    print(gr_rad)
    ax.scatter(t, z, c=gr_rad, marker='o', 
                  s=100/4, edgecolor='black', cmap='gray_r', 
                  vmin=glm.energy_min, vmax=glm.energy_max, 
                  zorder=100, label='groups')
        
    # events
    t = sec_since_basedate(flash_data.event_time_offset.data, panels.basedate)
    z = np.zeros_like(t) + ev_z
    ax.scatter(t, z+.1, c=ev_rad, marker='s', 
                    s=16/4, edgecolor='black', cmap='gray', 
                    vmin=glm.energy_min, vmax=glm.energy_max, 
                    zorder=101, label='events')

def plot_glm_b4d(glm, panels, flash_ids):
    for flash_data in gen_flashes(glm, flash_ids):
        plot_fge(flash_data, panels.panels['xy'], transform=transform_ll_to_map)
        plot_fge_tz(flash_data, panels.panels['tz'], panels.basedate, z=1.5)

In [19]:
plot_glm_b4d(glm, panels, flash_ids)

[ 45040.902  45040.904  45041.116  45041.176  45041.178  45041.192
  45041.21 ]
[ 1.5  1.5  1.5  1.5  1.5  1.5  1.5]
[  4.52596989e-15   4.52596989e-15   4.52596989e-15   1.06298498e-14
   6.05193987e-15   6.05193987e-15   4.52596989e-15]
[ 45052.74   45052.83   45052.88   45052.906  45052.91   45052.912
  45052.926  45052.928  45052.934  45052.936  45052.94   45052.942
  45052.944  45052.946  45052.948  45052.956  45052.958  45052.968
  45052.976  45052.996  45053.004  45053.018  45053.02   45053.028
  45053.034  45053.098  45053.112  45053.162  45053.192  45053.196
  45053.198  45053.204  45053.206  45053.228]
[ 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5
  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5
  1.5  1.5  1.5  1.5]
[  7.57790985e-15   6.05193987e-15   4.52596989e-15   1.06298498e-14
   7.57790985e-15   1.67337297e-14   6.05193987e-15   6.05193987e-15
   1.36817898e-14   4.52596989e-15   1.67337297e-14   3.35193996e-14

## FED Gridding

In [20]:
import subprocess, glob
from datetime import datetime
import os

from glmtools.grid.make_grids import grid_GLM_flashes
# start_time = datetime(2017,4,26, 12, 30, 0)
# end_time = datetime(2017,4,26, 12, 32, 0)
# glm_filenames = [filename]


start_time = datetime(2017,4,29, 8, 0, 0)
end_time = datetime(2017,4,29, 8, 10, 0)
glm_filenames = glob.glob('/data/LCFA-production/GLM-L2-LCFA_G16_s20170429/OR_GLM-L2-LCFA_G16_s2017119080*.nc') #[0:10]

grid_dir = os.path.join('/data/LCFA-production/', 'grid')
date = datetime(start_time.year, start_time.month, start_time.day)

from lmatools.grid.make_grids import write_cf_netcdf_latlon, dlonlat_at_grid_center, grid_h5flashfiles

outpath = grid_dir+'/20%s' %(date.strftime('%y/%b/%d'))
if os.path.exists(outpath) == False:
    os.makedirs(outpath)
    subprocess.call(['chmod', 'a+w', outpath, grid_dir+'/20%s' %(date.strftime('%y/%b')), grid_dir+'/20%s' %(date.strftime('%y'))])

# center_ID='WTLMA'
ctr_lat = 33.5
ctr_lon = -101.5
dx_km=10.0e3
dy_km=10.0e3
x_bnd_km = (-400.0e3, 400.0e3)
y_bnd_km = (-400.0e3, 400.0e3)
frame_interval = 60.0



dx, dy, x_bnd, y_bnd = dlonlat_at_grid_center(ctr_lat, ctr_lon, 
                            dx=dx_km, dy=dy_km,
                            x_bnd = x_bnd_km, y_bnd = y_bnd_km )
print(x_bnd, y_bnd)

grid_GLM_flashes(glm_filenames, start_time, end_time, proj_name='latlong',
        base_date = date, do_3d=False,
        dx=dx, dy=dy, frame_interval=frame_interval,
        #dz=dz, z_bnd=z_bnd_km,
        x_bnd=x_bnd, y_bnd=y_bnd, 
        ctr_lat=ctr_lat, ctr_lon=ctr_lon, outpath = outpath,
        output_writer = write_cf_netcdf_latlon, #output_writer_3d = write_cf_netcdf_3d_latlon,
        output_filename_prefix='GLM', spatial_scale_factor=1.0
        )

# lma_h5_filenames = glob.glob('/data/LCFA-production/LMArealtime/flashsort/h5_files/2017/Apr/29/LYLOUT_170429_0[6-7]*.h5')
# grid_h5flashfiles(lma_h5_filenames, start_time, end_time, proj_name='latlong',
#         base_date = date, do_3d=False,
#         dx=dx, dy=dy, frame_interval=frame_interval,
#         x_bnd=x_bnd, y_bnd=y_bnd,
#         #dz=dz, z_bnd=z_bnd_km,
#         ctr_lon=ctr_lon, ctr_lat=ctr_lat, outpath = outpath,
#         output_writer = write_cf_netcdf_latlon, #output_writer_3d = write_cf_netcdf_3d_latlon,
#         output_filename_prefix='WTLMA', spatial_scale_factor=1.0
#         )


((-105.80465892027087, -97.195341079729133), (29.893608635213752, 37.106391364786248))
Processing /data/LCFA-production/GLM-L2-LCFA_G16_s20170429/OR_GLM-L2-LCFA_G16_s20171190800000_e20171190800200_c20171190800228.nc
with points numbering 10320
accumulating  10320 points for  event
with points numbering 145
accumulating  145 points for  init
extent with points numbering 10320  with weights None
accumulating  1187 points for  extent
extent with points numbering 10320  with weights area
accumulating  1187 points for  footprint
extent with points numbering 10320  with weights specific_energy
accumulating  1187 points for  specific_energy
extent with points numbering 10320  with weights area
accumulating  1187 points for  flashsize_std
extent with points numbering 10320  with weights total_energy
accumulating  1187 points for  total_energy
Processing /data/LCFA-production/GLM-L2-LCFA_G16_s20170429/OR_GLM-L2-LCFA_G16_s20171190800200_e20171190800400_c20171190800429.nc
with points numbering 83

KeyboardInterrupt: 

## Debug area

In [21]:
import xarray as xr

nc_fed = xr.open_dataset('/data/LCFA-production/grid/2017/Apr/29/GLM_20170429_070000_3600_1src_0.1076deg-dx_flash_extent.nc')
nc_afa = xr.open_dataset('/data/LCFA-production/grid/2017/Apr/29/GLM_20170429_070000_3600_1src_0.1076deg-dx_footprint.nc')

In [22]:
nc_fed.flash_extent[42,:,:]

<xarray.DataArray 'flash_extent' (lon: 80, lat: 80)>
array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])
Dimensions without coordinates: lon, lat
Attributes:
    units:         Count per 0.0 km^2 pixel per 1.0 min
    long_name:     LMA flash extent density
    grid_mapping:  crs

In [23]:
sl = slice(42, None, None)
has_areas = (nc_afa.flash_footprint[42,:,:] > 0).data
has_flash = (nc_fed.flash_extent[42,:,:] > 0).data
both = (has_areas & has_flash).sum()
only_fed = has_flash.sum()
print both, only_fed

577 577


In [24]:
lon_range = (-100., -98.)
lat_range = (33., 36.)
conflict_files = glob.glob('/data/LCFA-production/GLM-L2-LCFA_G16_s20170429/OR_GLM-L2-LCFA_G16_s20171190742*.nc')
for cf in conflict_files:
    glm = GLMDataset(cf)
    flash_data = glm.lonlat_subset(lon_range=lon_range, lat_range=lat_range)
    print(fix_unsigned(flash_data.flash_area)[:])


[  893.01605991   735.9180409    659.4919776    278.57477322   431.27526081
  1344.29376701   279.33296829  1174.00315374   275.69363195   579.72985598
   203.05854401   501.18084648  1342.62573785  1270.14228892   658.4305045
   203.05854401   576.69707569   277.51330012   201.08723682   354.09100243
   664.19278705   665.86081621   278.27149519   277.51330012   202.75526598
   589.13147488   675.11079609   734.70492879   279.33296829   579.57821697
   965.34786981   351.36150017   885.88902622   350.45166609   202.30034894
   504.82018283   202.60362697  1039.19606987   278.87805125   430.97198278
   201.54215387   735.31148484  1025.85183659  1264.07672834   277.51330012
   506.03329495   281.15263647   435.67279223   889.8316406    872.08987591
   201.84543189   358.33689484   501.48412451   275.84527096   201.08723682
   353.02952933   810.37279701   962.01181149  1353.54374689   276.45182702
   200.93559781  1116.68360627   279.48460731  1530.50647679   275.54199293
  1187.195748